# RTML Lab 01: Setup

In this lab, we'll set up the Python and CUDA environment that we'll use for the rest of the semester.

## Requirements

The recommended environment for your work is Ubuntu Linux 20.04 LTS.

You will most likely have the best experience with Ubuntu by installing it natively on your development workstation.

However, an alternative path is to run Ubuntu side-by-side with Microsoft Windows using the Windows Subsystem for Linux (WSL).

According to our surveys, most DS&AI students are not yet ready to switch to Ubuntu for 100% of their work and would like a more gentle introduction
to the use of Linux for software development. Therefore, in this tutorial, we assume you are running Windows.

### WSL

The first step is to install WSL according to [Microsoft's WSL installation instructions](https://docs.microsoft.com/en-us/windows/wsl/install-win10).

When it's time to download a Linux distribution from the Windows store, choose "Ubuntu 20.04."

### OpenSSH feature in Windows

If OpenSSH is not already enabled, go to the "Manage Optional Features" settings panel in Windows, select "OpenSSH client", and click "Install".

More detail is available at [Microsoft's documentation page for OpenSSH installation](https://docs.microsoft.com/en-us/windows-server/administration/openssh/openssh_install_firstuse#installing-openssh-from-the-settings-ui-on-windows-server-2019-or-windows-10-1809).

### VSCode

Visual Studio Code is a lightweight yet full featured cross platform IDE for software development that has recently caught up
in terms of capabilities and popularity with other popular IDEs for Python such as PyCharm. It is reputed to be easier to configure and use, also.
We'll give it a try this semester. Download and install VSCode from [the Visual Studio downloads page](https://code.visualstudio.com/download).

## WSL Python environment

As a starting point, let's set up Python under WSL.

In the lab manuals for RTML, when you see commands beginning with '\$' like this: 

    $ ls
    
it means that you should run the command `ls` at your local WSL prompt.

As a first step, you'll need to update Ubuntu's package lists, upgrade any that need upgrading, and then install Python 3 and the PIP package manager:

    $ sudo apt-get update
    $ sudo apt-get upgrade
    $ sudo apt-get install python3-pip

Then you should be able to run Python code:

    $ python3
    ...
    >>> print('Hello, world!')
    Hello, world!
    >>>
    
(Here the `>>>` prompt means the Python interpreter prompt.)

To install Python packages, use PIP:

    $ pip3 install torch
    ...
    $ python3
    ...
    >>> import torch
    >>> torch.__version__
    '1.7.1'
   
That was easy, right?

To work at a professional level with Python, you'll probably want to look at `virtualenv`, which allows you to
maintain separate Python environments with all dependencies for each of your projects separately.
If you like, take a look at [the Python docs on virtualenv](https://docs.python-guide.org/dev/virtualenvs/#lower-level-virtualenv) for more information.
Note that in Ubuntu, the commands are `python3` and `pip3`, not `python` and `pip`.

In any case, in RTML, we will do most of our work remotely with another technology, Docker, to isolate our projects, so we don't need to get into virtualenv right now.

## SSH setup for remote access to GPU server in WSL

Next, we will set up our environment for access to a remote GPU server via SSH (the Secure Shell protocol).

We'll begin with WSL then use the same configuration for our Windows SSH configuration.

We'll assume that your GPU server is behind a gateway.

At AIT, the gateway is `bazooka.cs.ait.ac.th`, and the GPU server is `puffer.cs.ait.ac.th`. I'll assume your username on both the
gateway and GPU server is `st123456`. If you are using a different gateway or server, replace these names with your
specific ones in the rest of the tutorial, and obviously, replace `st123456` with your username.

First, let's try connecting to the gateway:

    $ ssh st123456@bazooka.cs.ait.ac.th
    Password for st123456@bazooka.cs.ait.ac.th:
    ...
    st123456@bazooka:~$ [Control-D or "exit" to exit]

Next, we want to avoid having to type a password every time we log in to the remote server.
We will generate an RSA public/private keypair for SSH to allow login without a password.
If you already have an RSA public/private keypair for SSH, you can skip this step.

    $ ssh-keygen -t rsa
    Generating public/private rsa key pair.
    Enter file in which to save the key (/home/mdailey/.ssh/id_rsa) [ENTER]
    Enter passphrase (empty for no passphrase): [USE A PASSPHRASE YOU'LL NEVER FORGET]
    Enter same passphrase again:
    Your identification has been saved in /home/mdailey/.ssh/id_rsa
    Your public key has been saved in /home/mdailey/.ssh/id_rsa.pub
    The key fingerprint is:
    SHA256:AgTtgfplWmns7Z0bQBOuYOawrKff0zZiI4rOVOVLHww mdailey@LAPTOP-NE58KA3C
    The key's randomart image is:
    +---[RSA 3072]----+
    |  .+. .          |
    |  ..o. .         |
    |..+o.E+          |
    |o* .@+o.         |
    |.o.O.+ooS        |
    |. + o +o.        |
    |...  + o..       |
    |+o..= = o.       |
    |==.o.= ...       |
    +----[SHA256]-----+
    $

Next, we copy the PUBLIC key to the server and tell the server to accept our login using the corresponding private key:

    $ scp .ssh/id_rsa.pub st123456@bazooka.cs.ait.ac.th:
    Password for st123456@bazooka.cs.ait.ac.th:
    ...
    $ ssh st123456@bazooka.cs.ait.ac.th
    Password for st123456@bazooka.cs.ait.ac.th:
    ...
    bazooka$ mkdir -p .ssh
    bazooka$ cat id_rsa.pub >> .ssh/authorized_keys
    bazooka$ exit
    $ ssh st123456@bazooka.cs.ait.ac.th
    Enter passphrase for key '/home/mdailey/.ssh/id_rsa': [USE THAT PASSPHRASE YOU'LL NEVER FORGET]
    ...
    bazooka$ exit
    $ 

We don't have to enter our password for the remote server anymore, but we still have to enter our passphrase for the key file. To fix that, we need something called the SSH Agent!
It's a little program that runs in the background, reads your private keys into memory (if you ask it to), and then later supplies your keys to SSH every time authentication is needed.

    $ eval `ssh-agent`
    $ ssh-add ~/.ssh/id_rsa
    Enter passphrase for /home/mdailey/.ssh/id_rsa:
    Identity added: /home/mdailey/.ssh/id_rsa (mdailey@LAPTOP-NE58KA3C)
    $ ssh st123456@bazooka.cs.ait.ac.th
    ...
    bazooka$ exit

OK! Now that we can jump to the gateway without a password, let's use it to jump to the GPU server. For this, you need a file `~/.ssh/config` with contents

    Host puffer
      Hostname puffer.cs.ait.ac.th
      ProxyCommand ssh bazooka.cs.ait.ac.th -W %h:%p
      User st123456
      ForwardAgent yes
  
If everything is OK, you should now be able to SSH directly to the GPU server without using passwords or passphrases:

    $ ssh puffer
    puffer$ exit

That was a lot of steps, but not too bad, right?

## SSH setup for remote access to GPU server in Windows

Now that everything is set up in WSL, it's easy to get it working in Windows directly. VSCode will use Windows' SSH client to connect to our GPU server, so that's why we need to set up
both.

Within WSL, copy your RSA key files and remote host configuration to your Windows home directory:

    $ mkdir -p /mnt/c/Users/Matthew\ Dailey/.ssh
    $ cp ~/.ssh/config /mnt/c/Users/Matthew\ Dailey/.ssh/
    $ cp ~/.ssh/id_rsa* /mnt/c/Users/Matthew\ Dailey/.ssh/

In the Windows Powershell RUNNING AS ADMINISTRATOR, tell Windows to always start the SSH Agent:

    PS C:\WINDOWS/system32> Set-Service ssh-agent -StartupType Automatic
    PS C:\WINDOWS/system32> Start-Service ssh-agent
    PS C:\WINDOWS/system32> Get-Service ssh-agent
    
    Status   Name               DisplayName
    ------   ----               -----------
    Running  ssh-agent          OpenSSH Authentication Agent

    PS C:\WINDOWS/system32> exit

Now, in an ordinary Powershell, you should be able to log in to the GPU server:

    PS C:\Users\Matthew Dailey> ssh puffer
    ...
    puffer$


## Testing NVIDIA/Docker integration

If you have docker version 19.03 or later installed on the server, you don't need to use a special program to access NVIDIA GPUs within docker. Try the following:

    puffer$ docker run -it --rm --gpus all ubuntu nvidia-smi

Once the docker images download, you should see some information about the GPU environment on the system. Try the following for a detailed list of GPUs you have access to:

    puffer$ docker run -it --rm --gpus all ubuntu nvidia-smi -L
    Fri Jan 15 00:58:18 2021
    +-----------------------------------------------------------------------------+
    | NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: N/A      |
    |-------------------------------+----------------------+----------------------+
    | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
    | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
    |                               |                      |               MIG M. |
    |===============================+======================+======================|
    |   0  GeForce RTX 208...  Off  | 00000000:84:00.0 Off |                  N/A |
    | 24%   38C    P0    56W / 250W |      0MiB / 11019MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   1  GeForce RTX 208...  Off  | 00000000:85:00.0 Off |                  N/A |
    | 22%   40C    P0    47W / 250W |      0MiB / 11019MiB |      1%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   2  GeForce RTX 208...  Off  | 00000000:88:00.0 Off |                  N/A |
    | 23%   36C    P0    51W / 250W |      0MiB / 11019MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    |   3  GeForce RTX 208...  Off  | 00000000:89:00.0 Off |                  N/A |
    | 31%   36C    P0    25W / 250W |      0MiB / 11019MiB |      0%      Default |
    |                               |                      |                  N/A |
    +-------------------------------+----------------------+----------------------+
    
    +-----------------------------------------------------------------------------+
    | Processes:                                                                  |
    |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
    |        ID   ID                                                   Usage      |
    |=============================================================================|
    |  No running processes found                                                 |
    +-----------------------------------------------------------------------------+
    puffer$

One fun trick is to use the DIGITS web application for a deep learning experiment:

    puffer$ docker run -itd --gpus all -p 5000:5000 nvidia/digits

You'll have to pick an unused port. Once running, check that the server is actually up:

    puffer$ wget http://localhost:5000

If successful, you're up. To forward local port 5000 to port 5000 on the GPU server, you need to modify the command you use to ssh to the server:

    $ ssh -L 5000:localhost:5000 puffer
    puffer$

Now you should be able to access http://localhost:5000 in your Web browser and access the DIGITS application running on the server.

## Build your docker image

To build a Docker image, you'll need to access the Internet. Unfortunately, on the GPU server,
we require a proxy server to access the Internet. To set a proxy with CSH, edit ~/.cshrc and put this at the end of the file:

setenv http_proxy http://192.41.170.23:3128
setenv https_proxy http://192.41.170.23:3128

For BASH, you would edit ~/.bashrc and place the following at the end of the file:

export http_proxy=http://192.41.170.23:3128
export https_proxy=http://192.41.170.23:3128

Now, let's prepare the docker environment on the GPU server

    $ scp ~/.ssh/id_rsa.pub puffer:
    $ ssh puffer
    puffer$ mkdir -p lab
    puffer$ mv id_rsa.pub lab/
    puffer$ cd lab
    puffer$ cat > Dockerfile <<EOF
    FROM nvidia/cuda:10.1-base

    ENV http_proxy http://192.41.170.23:3128
    ENV https_proxy http://192.41.170.23:3128

    RUN apt-get update && apt-get upgrade -y && apt-get install -y openssh-server
    RUN mkdir /var/run/sshd
    RUN mkdir /root/.ssh/
    COPY id_rsa.pub /root/.ssh/authorized_keys
    EXPOSE 22

    # set the locale to en_US.UTF-8
    RUN apt-get install -y locales
    ENV DEBIAN_FRONTEND noninteractive
    RUN echo "en_US.UTF-8 UTF-8" > /etc/locale.gen \
        && locale-gen en_US.UTF-8 \
        && dpkg-reconfigure locales \
        && /usr/sbin/update-locale LANG=en_US.UTF-8
    ENV LC_ALL en_US.UTF-8

    RUN apt-get install -y python3-pip
    RUN pip3 install numpy torch torchvision

    CMD ["/usr/sbin/sshd", "-D"]
    EOF
    puffer$ docker build . -t matt-lab1   # Use your own tag here!

If the docker image builds successfully, you're ready to go! Run the image as

    puffer$ docker run -p 2222:22 --gpus all matt-lab1

You'll want to use a unique port name in place of 2222 if there are others using the GPU server. If that works, you should be able to run

    puffer$ ssh root@localhost -p 2222

(Use whatever port you selected above in place of 2222, of course.) If that worked, now you can ssh to the gpu server and open a local port to the remote system:

    $ ssh -L 2222:localhost:2222 puffer "docker run -p 2222:22 --gpus all matt-lab1"

The first 2222 is the port on the LOCAL machine. There won't be any conflict there, so you can use what you like. The second port is the port on the REMOTE machine. It should match the port used on the GPU machine for the docker image's SSH port (also 2222 in the example above).

If all that worked, now your Docker container is up and running your image with GPU support, and local port 2222 is forwarded to the SSH port of the new Docker container. Now you should be able to run (on your local system)

    $ ssh root@localhost -p 2222
    1f7a17f71d25# pip3 list

You should see torch and numpy among the list of installed Python packages. All is good!

## Connect VSCode to your container on the GPU server

Now we are almost done. The last step is to tell VSCode to use the remote environment inside the new Docker container rather than the local system for running our Python code.

Use the Remote SSH extension in VSCode to connect with your container on the server.

To test that all is well, create a Python script and enter the code

    import torch
    torch.has_cuda

If you get the result True, you're successful!

## Try out an AlexNet model

Try some sample code executing the AlexNet model in PyTorch:

    import torch
    import urllib
    import os

    os.environ['http_proxy'] = 'http://192.41.170.23:3128'
    os.environ['https_proxy'] = 'http://192.41.170.23:3128'

    model = torch.hub.load('pytorch/vision:v0.5.0', 'alexnet', pretrained=True)

    model.eval()

    # Download an example image from the pytorch website

    url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
    try:
        urllib.URLopener().retrieve(url, filename)
    except:
        urllib.request.urlretrieve(url, filename)

    from PIL import Image
    from torchvision import transforms
    input_image = Image.open(filename)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #print(output[0])

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    softmax_scores = torch.nn.functional.softmax(output[0], dim=0)

    maxval, maxindex = output.max(1)
    print('Maximum value', maxval, 'at index', maxindex)

I got an evaluation time of 24 ms for CPU and 2 ms for CUDA on puffer.

## The independent part

OK, so now your goal is to try to train and evaluate the PyTorch AlexNet model on the CIFAR-10 dataset. You can use Torch's torchvision module to load the data into PyTorch tensors.

## The report

Your lab report should have the following sections:

1. Introduction: the background and goals of the lab
1. Methods: what you did, what parameters you tried, and so on
1. Results: what were the results
1. Conclusion: what did you learn from the lab, and what might be the next steps

In the results section, be sure to show training and validation loss as a function of training epochs. You'll also want to show results on a separate test set and give some analysis of the errors the classifier makes on the test set.